# CityBikes

Send a request to CityBikes for the city of your choice. 

In [1]:
#create an api request to citybik.es to get the information from Hamilton Ontario
import requests
import json
import pandas as pd

In [2]:
#make the request
response = requests.get('https://api.citybik.es/v2/networks')

#convert the response to a json object
data = response.json()

#convert the json object to a pandas dataframe
citybikes_df = pd.DataFrame(data['networks'])

In [3]:
citybikes_df.head()

,company,href,id,location,name,source,gbfs_href,license,ebikes
0,[ЗАО «СитиБайк»],/v2/networks/velobike-moscow,velobike-moscow,"{'city': 'Moscow', 'country': 'RU', 'latitude'...",Velobike,NaN,NaN,NaN,NaN
1,[Comunicare S.r.l.],/v2/networks/bicincitta-siena,bicincitta-siena,"{'city': 'Siena', 'country': 'IT', 'latitude':...",Bicincittà,https://www.bicincitta.com/frmLeStazioni.aspx?...,NaN,NaN,NaN
2,[Cyclopolis Systems],/v2/networks/cyclopolis-maroussi,cyclopolis-maroussi,"{'city': 'Maroussi', 'country': 'GR', 'latitud...",Cyclopolis,NaN,NaN,NaN,NaN
3,[Cyclopolis Systems],/v2/networks/cyclopolis-nafplio,cyclopolis-nafplio,"{'city': 'Nafplio', 'country': 'GR', 'latitude...",Cyclopolis,NaN,NaN,NaN,NaN
4,[Comunicare S.r.l.],/v2/networks/bicincitta-parco-dei-colli-di-ber...,bicincitta-parco-dei-colli-di-bergamo,"{'city': 'Parco dei Colli di Bergamo', 'countr...",Bicincittà,https://www.bicincitta.com/frmLeStazioni.aspx?...,NaN,NaN,NaN


In [4]:
#json normalization of the location column
location = pd.json_normalize(citybikes_df['location'])
location.columns = ['location.' + str(col) for col in location.columns]
citybikes_df = citybikes_df.drop('location', axis=1)
citybikes_df = pd.concat([citybikes_df, location], axis=1)


In [5]:
citybikes_df.head()

,company,href,id,name,source,gbfs_href,license,ebikes,location.city,location.country,location.latitude,location.longitude
0,[ЗАО «СитиБайк»],/v2/networks/velobike-moscow,velobike-moscow,Velobike,NaN,NaN,NaN,NaN,Moscow,RU,55.750000,37.616667
1,[Comunicare S.r.l.],/v2/networks/bicincitta-siena,bicincitta-siena,Bicincittà,https://www.bicincitta.com/frmLeStazioni.aspx?...,NaN,NaN,NaN,Siena,IT,43.318600,11.330600
2,[Cyclopolis Systems],/v2/networks/cyclopolis-maroussi,cyclopolis-maroussi,Cyclopolis,NaN,NaN,NaN,NaN,Maroussi,GR,38.056872,23.808330
3,[Cyclopolis Systems],/v2/networks/cyclopolis-nafplio,cyclopolis-nafplio,Cyclopolis,NaN,NaN,NaN,NaN,Nafplio,GR,37.563940,22.809340
4,[Comunicare S.r.l.],/v2/networks/bicincitta-parco-dei-colli-di-ber...,bicincitta-parco-dei-colli-di-bergamo,Bicincittà,https://www.bicincitta.com/frmLeStazioni.aspx?...,NaN,NaN,NaN,Parco dei Colli di Bergamo,IT,45.722956,9.649230


In [6]:
#filter the dataframe to only show the information for 'Hamilton, ON' in the location.city column
hamilton_bike_companies = citybikes_df[citybikes_df['location.city'] == 'Hamilton, ON']
hamilton_bike_companies.head()

,company,href,id,name,source,gbfs_href,license,ebikes,location.city,location.country,location.latitude,location.longitude
74,[Social Bicycles Inc.],/v2/networks/sobi-hamilton,sobi-hamilton,SoBi,NaN,https://hamilton.socialbicycles.com/opendata/g...,NaN,NaN,"Hamilton, ON",CA,43.256436,-79.869297


In [7]:
#create a new request to get the information for the Hamilton Social Bicycles inc. api using the link from the gbfs_href column
#its suprising to me that there is only one company in Hamilton for bike share
response = requests.get(hamilton_bike_companies['gbfs_href'].values[0])
data = response.json()
sobi_df = pd.DataFrame(data['data']['en']['feeds'])
sobi_df

,name,url
0,gbfs,http://hamilton.socialbicycles.com/opendata/gb...
1,system_information,http://hamilton.socialbicycles.com/opendata/sy...
2,station_information,http://hamilton.socialbicycles.com/opendata/st...
3,station_status,http://hamilton.socialbicycles.com/opendata/st...
4,free_bike_status,http://hamilton.socialbicycles.com/opendata/fr...
5,system_hours,http://hamilton.socialbicycles.com/opendata/sy...
6,system_calendar,http://hamilton.socialbicycles.com/opendata/sy...
7,system_regions,http://hamilton.socialbicycles.com/opendata/sy...
8,system_pricing_plans,http://hamilton.socialbicycles.com/opendata/sy...
9,system_alerts,http://hamilton.socialbicycles.com/opendata/sy...


In [8]:
#create a new request to grab station information from the station_information url
response = requests.get(sobi_df[sobi_df['name'] == 'station_information']['url'].values[0])
data = response.json()
station_info = pd.DataFrame(data['data']['stations'])
station_info.head()

,station_id,name,region_id,lon,lat,address,rental_methods
0,hub_435,Gore Park,region_86,-79.868664,43.256317,"20 King Street East, Hamilton","[KEY, APPLEPAY, ANDROIDPAY, TRANSITCARD, ACCOU..."
1,hub_436,James North at Mulberry,region_86,-79.866743,43.261819,"200 James Street North, Hamilton","[KEY, APPLEPAY, ANDROIDPAY, TRANSITCARD, ACCOU..."
2,hub_527,Seedworks,region_86,-79.863613,43.259111,"126 Catharine Street North, Hamilton","[KEY, APPLEPAY, ANDROIDPAY, TRANSITCARD, ACCOU..."
3,hub_530,Cootes at York and King,region_86,-79.949656,43.265581,"10 Cootes Drive, Hamilton","[KEY, APPLEPAY, ANDROIDPAY, TRANSITCARD, ACCOU..."
4,hub_535,Osler at Grant,region_86,-79.942643,43.259550,"89-101 Osler Drive, Hamilton","[KEY, APPLEPAY, ANDROIDPAY, TRANSITCARD, ACCOU..."


In [9]:
#create a new request to grab station status information from the station_status url
response = requests.get(sobi_df[sobi_df['name'] == 'station_status']['url'].values[0])
data = response.json()
station_status = pd.DataFrame(data['data']['stations'])
station_status.head()

,station_id,num_bikes_available,num_bikes_disabled,num_docks_available,is_installed,is_renting,is_returning,last_reported
0,hub_435,4,0,12,1,1,1,1709996786
1,hub_436,3,0,15,1,1,1,1709996786
2,hub_527,6,0,1,1,1,1,1709996786
3,hub_530,11,0,4,1,1,1,1709996786
4,hub_535,5,0,7,1,1,1,1709996786


In [10]:
#create a new request to grab free bike information from the free_bike_status url
response = requests.get(sobi_df[sobi_df['name'] == 'free_bike_status']['url'].values[0])
data = response.json()
free_bike_status = pd.DataFrame(data['data']['bikes'])
free_bike_status.head()


,bike_id,name,lon,lat,is_reserved,is_disabled
0,bike_2135,369-Jillian,-79.863928,43.268858,0,0
1,bike_38506,908-The Meg,-79.830207,43.257560,0,0
2,bike_2015,254-Dallas,-79.849585,43.264480,0,0
3,bike_3225,721-Arabella,-79.880798,43.249728,0,0
4,bike_2290,532-Delfina,-79.912643,43.260928,0,0


Parse through the response to get the details you want for the bike stations in that city (latitude, longitude, number of bikes). 

In [11]:
#join the station_info and station_status dataframes on the station_id column
sobi_station_info = pd.merge(station_info, station_status, on='station_id')
sobi_station_info.head()

,station_id,name,region_id,lon,lat,address,rental_methods,num_bikes_available,num_bikes_disabled,num_docks_available,is_installed,is_renting,is_returning,last_reported
0,hub_435,Gore Park,region_86,-79.868664,43.256317,"20 King Street East, Hamilton","[KEY, APPLEPAY, ANDROIDPAY, TRANSITCARD, ACCOU...",4,0,12,1,1,1,1709996786
1,hub_436,James North at Mulberry,region_86,-79.866743,43.261819,"200 James Street North, Hamilton","[KEY, APPLEPAY, ANDROIDPAY, TRANSITCARD, ACCOU...",3,0,15,1,1,1,1709996786
2,hub_527,Seedworks,region_86,-79.863613,43.259111,"126 Catharine Street North, Hamilton","[KEY, APPLEPAY, ANDROIDPAY, TRANSITCARD, ACCOU...",6,0,1,1,1,1,1709996786
3,hub_530,Cootes at York and King,region_86,-79.949656,43.265581,"10 Cootes Drive, Hamilton","[KEY, APPLEPAY, ANDROIDPAY, TRANSITCARD, ACCOU...",11,0,4,1,1,1,1709996786
4,hub_535,Osler at Grant,region_86,-79.942643,43.259550,"89-101 Osler Drive, Hamilton","[KEY, APPLEPAY, ANDROIDPAY, TRANSITCARD, ACCOU...",5,0,7,1,1,1,1709996786


In [12]:
#filter the sobi_station_info dataframe to only show the stations that have bikes available and save the result to a new dataframe
available_bikes = sobi_station_info[sobi_station_info['num_bikes_available'] > 0]
available_bikes.head()

,station_id,name,region_id,lon,lat,address,rental_methods,num_bikes_available,num_bikes_disabled,num_docks_available,is_installed,is_renting,is_returning,last_reported
0,hub_435,Gore Park,region_86,-79.868664,43.256317,"20 King Street East, Hamilton","[KEY, APPLEPAY, ANDROIDPAY, TRANSITCARD, ACCOU...",4,0,12,1,1,1,1709996786
1,hub_436,James North at Mulberry,region_86,-79.866743,43.261819,"200 James Street North, Hamilton","[KEY, APPLEPAY, ANDROIDPAY, TRANSITCARD, ACCOU...",3,0,15,1,1,1,1709996786
2,hub_527,Seedworks,region_86,-79.863613,43.259111,"126 Catharine Street North, Hamilton","[KEY, APPLEPAY, ANDROIDPAY, TRANSITCARD, ACCOU...",6,0,1,1,1,1,1709996786
3,hub_530,Cootes at York and King,region_86,-79.949656,43.265581,"10 Cootes Drive, Hamilton","[KEY, APPLEPAY, ANDROIDPAY, TRANSITCARD, ACCOU...",11,0,4,1,1,1,1709996786
4,hub_535,Osler at Grant,region_86,-79.942643,43.259550,"89-101 Osler Drive, Hamilton","[KEY, APPLEPAY, ANDROIDPAY, TRANSITCARD, ACCOU...",5,0,7,1,1,1,1709996786


In [13]:
#remove region_id , is_installed, is_renting, is_returning, rental_methods, and num_docks_available columns from the available_bikes dataframe
available_bikes = available_bikes.drop(['region_id', 'is_installed', 'is_renting', 'is_returning','rental_methods', 'num_docks_available'], axis=1)
available_bikes.head()

,station_id,name,lon,lat,address,num_bikes_available,num_bikes_disabled,last_reported
0,hub_435,Gore Park,-79.868664,43.256317,"20 King Street East, Hamilton",4,0,1709996786
1,hub_436,James North at Mulberry,-79.866743,43.261819,"200 James Street North, Hamilton",3,0,1709996786
2,hub_527,Seedworks,-79.863613,43.259111,"126 Catharine Street North, Hamilton",6,0,1709996786
3,hub_530,Cootes at York and King,-79.949656,43.265581,"10 Cootes Drive, Hamilton",11,0,1709996786
4,hub_535,Osler at Grant,-79.942643,43.259550,"89-101 Osler Drive, Hamilton",5,0,1709996786


In [14]:
#update the last_reported column to be a datetime object
available_bikes['last_reported'] = pd.to_datetime(available_bikes['last_reported'], unit='s')
available_bikes.head()

,station_id,name,lon,lat,address,num_bikes_available,num_bikes_disabled,last_reported
0,hub_435,Gore Park,-79.868664,43.256317,"20 King Street East, Hamilton",4,0,2024-03-09 15:06:26
1,hub_436,James North at Mulberry,-79.866743,43.261819,"200 James Street North, Hamilton",3,0,2024-03-09 15:06:26
2,hub_527,Seedworks,-79.863613,43.259111,"126 Catharine Street North, Hamilton",6,0,2024-03-09 15:06:26
3,hub_530,Cootes at York and King,-79.949656,43.265581,"10 Cootes Drive, Hamilton",11,0,2024-03-09 15:06:26
4,hub_535,Osler at Grant,-79.942643,43.259550,"89-101 Osler Drive, Hamilton",5,0,2024-03-09 15:06:26


Put your parsed results into a DataFrame.

In [15]:
available_bikes

,station_id,name,lon,lat,address,num_bikes_available,num_bikes_disabled,last_reported
0,hub_435,Gore Park,-79.868664,43.256317,"20 King Street East, Hamilton",4,0,2024-03-09 15:06:26
1,hub_436,James North at Mulberry,-79.866743,43.261819,"200 James Street North, Hamilton",3,0,2024-03-09 15:06:26
2,hub_527,Seedworks,-79.863613,43.259111,"126 Catharine Street North, Hamilton",6,0,2024-03-09 15:06:26
3,hub_530,Cootes at York and King,-79.949656,43.265581,"10 Cootes Drive, Hamilton",11,0,2024-03-09 15:06:26
4,hub_535,Osler at Grant,-79.942643,43.259550,"89-101 Osler Drive, Hamilton",5,0,2024-03-09 15:06:26
...,...,...,...,...,...,...,...,...
138,hub_5845,Jackson Square,-79.869577,43.256935,"2-44 King Street West, Hamilton",8,0,2024-03-09 15:06:26
139,hub_5846,Cannon at Steven,-79.849981,43.256461,"400-408 Cannon Street East, Hamilton",1,0,2024-03-09 15:06:26
141,hub_5903,Pier 8,-79.856907,43.275289,"595 Catharine Street North, Hamilton",2,0,2024-03-09 15:06:26
142,hub_6148,King William at James,-79.868487,43.257635,"39 King William Street, Hamilton",2,0,2024-03-09 15:06:26


In [16]:
#order the available_bikes dataframe by the num_bikes_available column in descending order
available_bikes = available_bikes.sort_values(by='num_bikes_available', ascending=False)
available_bikes

,station_id,name,lon,lat,address,num_bikes_available,num_bikes_disabled,last_reported
9,hub_540,Sanders at Hollywood,-79.926481,43.259903,"26, Sanders Boulevard, University Gardens, Ham...",27,0,2024-03-09 15:06:26
14,hub_545,Emerson at Whitney,-79.922216,43.252142,"202, Emerson Street, University Gardens, Hamil...",16,0,2024-03-09 15:06:26
26,hub_557,Dundurn at Aberdeen,-79.892656,43.251458,"330-346 Dundurn Street South, Hamilton",13,0,2024-03-09 15:06:26
39,hub_571,Oxford at York,-79.877794,43.264564,"40-48 Oxford Street, Hamilton",13,0,2024-03-09 15:06:26
23,hub_554,Macklin at King,-79.896498,43.262167,"768, King Street West, McMaster Innovation Par...",12,0,2024-03-09 15:06:26
...,...,...,...,...,...,...,...,...
69,hub_605,Wentworth at King,-79.849104,43.251990,"16 Wentworth Street South, Hamilton",1,0,2024-03-09 15:06:26
137,hub_5838,King at Sanford,-79.846358,43.251884,"735, King Street East, Hamilton, Ontario",1,0,2024-03-09 15:06:26
97,hub_894,Forsyth at Sterling,-79.915806,43.262714,"88 Forsyth Avenue North, Hamilton",1,0,2024-03-09 15:06:26
139,hub_5846,Cannon at Steven,-79.849981,43.256461,"400-408 Cannon Street East, Hamilton",1,0,2024-03-09 15:06:26


In [17]:
#save the available_bikes dataframe to a csv file
available_bikes.to_csv('..\\data\\available_bikes.csv', index=False)